In [9]:
import operator
import numpy as np
training_data = [
   #'Outlook', 'Humidity', 'Wind', 'Play/NotPlay'
    ['Sunny', 'High', 'Weak', 0],
    ['Sunny', 'High', 'Strong', 0],
    ['Overcast', 'High', 'Weak', 1],
    ['Rain', 'High', 'Weak', 1],
    ['Rain', 'Normal', 'Weak', 1],
    ['Rain', 'Normal', 'Strong', 0],
    ['Overcast', 'Normal', 'Strong', 1],
    ['Sunny', 'High', 'Weak', 0],
    ['Sunny', 'Normal', 'Weak', 1],
    ['Rain', 'Normal', 'Weak', 1],
    ['Sunny', 'Normal', 'Strong', 1],
    ['Overcast', 'High', 'Strong', 1],
    ['Overcast', 'Normal', 'Weak', 1]
]
test_data = [
    ['Rain', 'High', 'Strong']       #True Class = 0
]

In [10]:
def is_numeric(value):
    return isinstance(value, int) or isinstance(value, float)

def get_answer_abs(row, feature, value):
    if row[feature] == value:
        return True
    else:
        return False
def get_answer_num(row, feature, value):
    if row[feature] >= value:
        return True
    else:
        return False

def get_split_rows(data, feature, value):
    left_rows = []
    right_rows = []
    for r in data:
        if is_numeric(value):
            answer = get_answer_num(r, feature, value)
        else:
            answer = get_answer_abs(r, feature, value)
        if answer:
            left_rows.append(r)
        else:
            right_rows.append(r)
    return left_rows, right_rows

l_rows, r_rows = get_split_rows(training_data, 1, 'High' )
print(l_rows, r_rows, sep='\n\n')


[['Sunny', 'High', 'Weak', 0], ['Sunny', 'High', 'Strong', 0], ['Overcast', 'High', 'Weak', 1], ['Rain', 'High', 'Weak', 1], ['Sunny', 'High', 'Weak', 0], ['Overcast', 'High', 'Strong', 1]]

[['Rain', 'Normal', 'Weak', 1], ['Rain', 'Normal', 'Strong', 0], ['Overcast', 'Normal', 'Strong', 1], ['Sunny', 'Normal', 'Weak', 1], ['Rain', 'Normal', 'Weak', 1], ['Sunny', 'Normal', 'Strong', 1], ['Overcast', 'Normal', 'Weak', 1]]


In [11]:
def class_count(rows):
    counts = {}
    for i in range(len(rows)):
        if rows[i][-1] not in counts.keys():
            counts[rows[i][-1]] = 1
        else:
            counts[rows[i][-1]] += 1
    return counts
def impurity(rows):
    counts = class_count(rows)
    impurity = 1
    for clas in counts:
        prob = counts[clas]/len(rows)
        impurity -= prob**2
    return impurity
def info_gain(left, right, data):
    p = float(len(left)) / (len(left) + len(right))
    return impurity(data) - p * impurity(left) - (1 - p) * impurity(right)

info_gain(l_rows, r_rows, training_data)

0.06339814032121721

In [12]:
features = ['Outlook', 'Humidity', 'Wind']
def find_unique_feat(col):
    unq = []
    for val in col:
        if val not in unq:
            unq.append(val)
        else:
            continue
    return unq

def find_best_spilt(data):
#     print('Starting Data', data, sep='\n')
    gains = []
    ques = []
    true_rows = []
    false_rows = []
    
    for feat in range(len(data[0])-1):
        unq = find_unique_feat([data[k][feat] for k in range(len(data))])
        for val in unq:
#             print('Is {} = {}'.format(features[feat], val))
            l_rows, r_rows = get_split_rows(data, feat, val)
            true_rows.append(l_rows)
            false_rows.append(r_rows)
            
            gain = info_gain(l_rows, r_rows, data)
#             print(gain)
            
            ques.append([feat, val])
            gains.append(gain)
            
    best_split = np.argmax(np.asarray(gains))
    return ques[best_split], gains[best_split], true_rows[best_split], false_rows[best_split]

ques, gain, true_rows, false_rows = find_best_spilt(training_data)
print(38*'-'+'Best Split'+38*'-',ques, gain, true_rows, false_rows, sep='\n')

--------------------------------------Best Split--------------------------------------
[0, 'Sunny']
0.10680473372781063
[['Sunny', 'High', 'Weak', 0], ['Sunny', 'High', 'Strong', 0], ['Sunny', 'High', 'Weak', 0], ['Sunny', 'Normal', 'Weak', 1], ['Sunny', 'Normal', 'Strong', 1]]
[['Overcast', 'High', 'Weak', 1], ['Rain', 'High', 'Weak', 1], ['Rain', 'Normal', 'Weak', 1], ['Rain', 'Normal', 'Strong', 0], ['Overcast', 'Normal', 'Strong', 1], ['Rain', 'Normal', 'Weak', 1], ['Overcast', 'High', 'Strong', 1], ['Overcast', 'Normal', 'Weak', 1]]


In [13]:
class LeafNode:
    def __init__(self, rows):
        counts = class_count(rows)
        self.pred = max(counts.items(), key=operator.itemgetter(1))[0]
class DecisionNode:
    def __init__(self, ques, l_branch, r_branch):
        self.true_branch = l_branch
        self.false_branch = r_branch
        self.question = ques

In [14]:
def make_tree(data):
    
    ques, gain, true_rows, false_rows = find_best_spilt(data)
    
    if gain == 0:
        return LeafNode(data)
    true_branch = make_tree(true_rows)
    false_branch = make_tree(false_rows)
    
    return DecisionNode(ques, true_branch, false_branch)
tree = make_tree(training_data)

In [15]:
def print_tree(node, spacing=""):
    
    if isinstance(node, LeafNode):
        print (spacing + "Predict", node.pred)
        return

    print (spacing + str(node.question))
    
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")
    
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")
print_tree(tree)

[0, 'Sunny']
--> True:
  [1, 'High']
  --> True:
    Predict 0
  --> False:
    Predict 1
--> False:
  [2, 'Weak']
  --> True:
    Predict 1
  --> False:
    [0, 'Rain']
    --> True:
      Predict 0
    --> False:
      Predict 1


In [16]:
def classify(row, node):
    if isinstance(node, LeafNode):
        return node.pred
    
    if get_answer_abs(row, node.question[0], node.question[1]):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)
classify(test_data[0], tree)

0

In [45]:
training_data = [
   #'Outlook', 'Humidity', 'Wind', 'Play/NotPlay'
    ['Sunny', 'High', 'Weak', 0],
    ['Sunny', 'High', 'Strong', 0],
    ['Overcast', 'High', 'Weak', 1],
    ['Rain', 'High', 'Weak', 1],
    ['Rain', 'Normal', 'Weak', 1],
    ['Rain', 'Normal', 'Strong', 0],
    ['Overcast', 'Normal', 'Strong', 1],
    ['Sunny', 'High', 'Weak', 0],
    ['Sunny', 'Normal', 'Weak', 1],
    ['Rain', 'Normal', 'Weak', 1],
    ['Sunny', 'Normal', 'Strong', 1],
    ['Overcast', 'High', 'Strong', 1],
    ['Overcast', 'Normal', 'Weak', 1]
]
test_data = [
    ['Rain', 'High', 'Strong']       #True Class = 0
]
num_trees = 10
trees = []
predictions = {
    '0' : 0,
    '1' : 0
}
for k in range(num_trees):
    data = []
    indexes = [np.random.randint(1, 13) for i in range(np.random.randint(5, 13))]
    for index in indexes:
        data.append(training_data[index])
    tree = make_tree(data)
    trees.append(tree)
for k in range(num_trees):
    y_pred = classify(training_data[3][:-1], trees[k])
    if y_pred == 0:
        predictions['0'] += 1
    elif y_pred == 1:
        predictions['1'] += 1
print(predictions)

{'1': 7, '0': 3}
